In [2]:
import pandas as pd
import numpy as np

In [3]:
prd_path=r"C:\Users\Felicio.Cassalho\work\Python_Development\ofs_dps\data\model\1d_node/073802511_stofs3d_salt_999_None_model.prd"
obs_path=r"C:\Users\Felicio.Cassalho\work\Python_Development\ofs_dps\data\observations\1d_station/073802511_stofs3d_salt_station.obs"

In [4]:
obs_df = pd.read_csv(obs_path,
                delim_whitespace=True,
                header=None,
                )

C:\Users\Felicio.Cassalho\AppData\Local\Temp\1\ipykernel_22648\2329384873.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  obs_df = pd.read_csv(obs_path,


In [5]:
ofs_df = pd.read_csv(prd_path,
                delim_whitespace=True,
                header=None,
                )

C:\Users\Felicio.Cassalho\AppData\Local\Temp\1\ipykernel_22648\1571357676.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ofs_df = pd.read_csv(prd_path,


In [6]:
obs_df["DateTime"] = pd.to_datetime(
        dict(
            year=obs_df[1],
            month=obs_df[2],
            day=obs_df[3],
            hour=obs_df[4],
            minute=obs_df[5],
        )
    )
obs_df = obs_df.rename(columns={6: "OBS"})

In [7]:
ofs_df["DateTime"] = pd.to_datetime(
    dict(
        year=ofs_df[1],
        month=ofs_df[2],
        day=ofs_df[3],
        hour=ofs_df[4],
        minute=ofs_df[5],
    )
)
ofs_df = ofs_df.rename(columns={6: "OFS"})

In [8]:
paired_0 = pd.DataFrame()
paired_0["DateTime"] = ofs_df["DateTime"]

In [9]:
# paired_0['fill']=-9999

In [10]:
paired_obs = pd.concat([paired_0, obs_df]).sort_values(
        by="DateTime"
    ) 

In [11]:
paired_obs = paired_obs[
    ~paired_obs["DateTime"].duplicated(keep=False)
    | paired_obs[["OBS"]].notnull().any(axis=1)
]

In [12]:
paired_obs = (
        paired_obs.sort_values(by="DateTime")
        .set_index("DateTime")
        .astype(float)
        .interpolate(method="time",limit=7,limit_direction='both')
        # .ffill()
        # .bfill()
        .reset_index()
    )

In [13]:
#paired_obs[8550:8600]

In [14]:
paired_ofs = pd.concat([paired_0, ofs_df]).sort_values(
        by="DateTime"
    )  

In [15]:
paired_ofs = paired_ofs[
    ~paired_ofs["DateTime"].duplicated(keep=False)
    | paired_ofs[["OFS"]].notnull().any(axis=1)
]
paired_ofs = (
    paired_ofs.sort_values(by="DateTime")
    .set_index("DateTime")
    .astype(float)
    .interpolate(method="time",limit=7,limit_direction='both')
    # .ffill()
    # .bfill()
    .reset_index()
)

In [16]:
paired = pd.concat([paired_obs, paired_ofs]).sort_values(by="DateTime")
paired = paired.groupby("DateTime", as_index=False).max()
# paired = paired.dropna().reset_index()

In [17]:
paired = paired[paired['DateTime'].isin(paired_ofs['DateTime'])]

In [18]:
paired["BIAS"] = paired["OFS"] - paired["OBS"]

In [28]:
type(np.array(paired_obs[8599:8600]["OBS"])[0])

numpy.float64

In [29]:
paired = paired.reset_index()

In [30]:
paired = paired.drop(columns=['index', 'DateTime'])

In [32]:
paired[8599:]

,0,1,2,3,4,5,OBS,OFS,BIAS
8599,240.722222,2021.0,8.0,29.0,17.000000,20.0,NaN,27.7041,NaN
8600,240.729167,2021.0,8.0,29.0,17.000000,30.0,NaN,27.8807,NaN
8601,240.736111,2021.0,8.0,29.0,17.000000,40.0,NaN,28.0714,NaN
8602,240.743056,2021.0,8.0,29.0,17.000000,50.0,NaN,28.2765,NaN
8603,240.750000,2021.0,8.0,29.0,18.000000,0.0,NaN,28.4938,NaN
...,...,...,...,...,...,...,...,...,...
13537,275.013889,2021.0,10.0,3.0,0.000000,20.0,3.033333,24.5022,21.468867
13538,275.020833,2021.0,10.0,3.0,0.000000,30.0,3.000000,24.4917,21.491700
13539,275.027778,2021.0,10.0,3.0,0.333333,40.0,3.033333,24.4824,21.449067
13540,275.034722,2021.0,10.0,3.0,0.666667,50.0,3.066667,24.4742,21.407533


In [34]:
xxx = paired.dropna()

In [38]:
xxx[8550:8600]

,0,1,2,3,4,5,OBS,OFS,BIAS
8550,240.381944,2021.0,8.000268,29.000000,9.000000,10.000000,10.997912,26.9051,15.907188
8551,240.388889,2021.0,8.000535,29.000000,9.000000,20.000000,10.995823,26.8922,15.896377
8552,240.395833,2021.0,8.000803,29.000000,9.000000,30.000000,10.993735,26.8787,15.884965
8553,240.402778,2021.0,8.001071,29.000000,9.000000,40.000000,10.991647,26.8657,15.874053
8554,240.409722,2021.0,8.001339,29.000000,9.000000,50.000000,10.989558,26.8543,15.864742
8555,240.416667,2021.0,8.001606,29.000000,10.000000,0.048193,10.987470,26.8440,15.856530
8556,240.423611,2021.0,8.001874,29.000000,10.000000,10.000000,10.985382,26.8344,15.849018
12277,266.263889,2021.0,9.000000,24.009371,7.003748,29.943775,3.214618,23.8224,20.607782
12278,266.270833,2021.0,9.000000,24.008032,7.003213,30.000000,3.212530,23.8299,20.617370
12279,266.277778,2021.0,9.000000,24.006693,7.002677,40.000000,3.210442,23.8369,20.626458
